<a href="https://colab.research.google.com/github/zhenyisx/scoutoid/blob/main/scraper2db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To scrape website and populate to database (csv file) on Google Drive. The files can then be uploaded to Goolge Storage for clients (i.e., stremlit) to consume.

# Import and Function Definitions

In [ ]:
# Imports
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import re

from google.colab import  drive
drive.mount('/drive')

%matplotlib inline

# TODO
# - home team and away team are reverse in current table
# - team should add city and state

In [ ]:
def get_scores_from_soup(maxpreps_day_soup, this_day):
  """to get scores of games in one day.

  Args:
      maxpreps_day_soup (soup): a soup parsed from MaxPreps HTML by BS.
      this_day (str): the date.

  Returns:
      schedulf_df (dataframe): the dataframe of all game results.

  """

  # create dateframe of game schedules and scores
  schedule_df = pd.DataFrame({'Date': pd.Series(dtype='datetime64[ns]'),
                    'Home Team': pd.Series(dtype='str'),
                    'Home Score': pd.Series(dtype='int'),
                    'Away Team': pd.Series(dtype='str'),
                    'Away Score': pd.Series(dtype='int')})

  # populate the datafrme by analyzing the soup (MaxPreps specific)
  # to use ChromeDev Tool to get the following rules of find 
  matches = maxpreps_day_soup.find('div', {'class': 'contests'})

  for m in matches.find_all('ul', {'class': 'teams'}):
    raw_record = m.find_all("li")

    # home score
    try:
      home_score = int(raw_record[0].find('div', {'class': 'score'}).text.strip())
    except ValueError:
        # Handle the exception
        print('score not valid, using -1 instead')
        home_score = -1
    except AttributeError:
        # Handle the exception
        print('score not valid, using -1 instead')
        home_score = -1
    # print(home_score)

    # home name
    home_name = raw_record[0].find('div', {'class': 'name'}).text.strip()
    # print(home_name)

    # away score
    try:
      away_score = int(raw_record[1].find('div', {'class': 'score'}).text.strip())
    except ValueError:
        # Handle the exception
        print('score not valid, using -1 instead')
        away_score = -1
    except AttributeError:
        # Handle the exception
        print('score not valid, using -1 instead')
        away_score = -1
    # print(away_score)

    # away name
    away_name = raw_record[1].find('div', {'class': 'name'}).text.strip()
    # print(away_name)

    m_result = {'Date':this_day, 'Home Team':home_name, 'Home Score':home_score, 
                'Away Team':away_name, 'Away Score':away_score}
    schedule_df = schedule_df.append(m_result, ignore_index=True)

  schedule_df = schedule_df.drop_duplicates(keep='last')

  return schedule_df
  

def get_scores_from_maxpreps_for_one_day(today):
  """to get scores of games from MaxPreps in one day.

  Example:
  df = get_scores_from_maxpreps_for_one_day("12/1/2022")
  print(df)

  Args:
      this_day (str): the date str in '%m/%d/%Y' format.

  Returns:
      schedulf_df (dataframe): the dataframe of all game results.

  """
  # maxpreps's homepage
  maxpreps_url = 'https://www.maxpreps.com/ca/central-coast-section/soccer/girls/scores/?date={}'.format(today)  # please change the date if needed

  # Use requests to retrieve data from a given URL
  maxpreps_response = requests.get(maxpreps_url, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

  # Parse the whole HTML page using BeautifulSoup
  maxpreps_soup = BeautifulSoup(maxpreps_response.text, 'html.parser')

  # Title of the parsed page
  print(maxpreps_soup.title.text)

  # check if the dates match
  try:
    content_date = datetime.strptime(maxpreps_soup.title.text.split('|')[1].strip().split()[0], '%m/%d/%Y')
    # another way to find the content current date
    # content = maxpreps_soup.find('div', {'class': 'calendar'}).find('ol', {'class': 'week'}).find('a', {'class': 'btn btn-default active'})
    # content_date = datetime.strptime(content.text, '%m/%d/%Y'))
    if content_date == datetime.strptime(today, '%m/%d/%Y'):
      print('Found Games for {}'.format(content_date))
      scores_df = get_scores_from_soup(maxpreps_soup, content_date)
      return scores_df
    else:
      print('No Games found for {}'.format(today))
  except TypeError:
    print('No Games found for {}'.format(today))
  except ValueError:
    print('No Games found for {}'.format(today))


def get_scores_from_maxpreps_for_range(start_date, end_date):
  """to get scores of games from MaxPreps in a date range.

  Example:
  start_date = date(2022, 11, 10)
  end_date = date(2023, 2, 10)
  scores = get_scores_from_maxpreps_for_range(start_date, end_date)

  Args:
      start_date (date): the start date.
      end_date (date): the end date.

  Returns:
      schedulf_df (dataframe): the dataframe of all game results.

  """
  def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

  dfs = []
  for single_date in daterange(start_date, end_date):
      today = single_date.strftime("%m/%d/%Y")
      scores_df = get_scores_from_maxpreps_for_one_day(today)
      if scores_df is None:
        print('No games found on {}'.format(today))
      else:
        print('Found {} games for {}'.format(len(scores_df), today))
      dfs.append(scores_df)

  # see pd.concat documentation for more info
  concat_scores_df = pd.concat(dfs)
  concat_scores_df = concat_scores_df.drop_duplicates(keep='last')
  return concat_scores_df


In [ ]:
def get_rankings_from_scores(schedule_df):
  """to ranking statistics of teams from the schedules and scores

  Example:
  df = get_scores_from_maxpreps_for_one_day("12/1/2022")
  print(df)

  Args:
      schedule_df (dataframe): the scores dataframe.

  Returns:
      ranking_df (dataframe): the dataframe of ranking statistics.

  """
  teams = list(set(list(schedule_df["Home Team"].unique()) + 
                   list(schedule_df["Away Team"].unique())))

  # initialize the ranking dataframe
  ranking_df = pd.DataFrame({
                    'Team': pd.Series(dtype='str'),
                    'Total Points': pd.Series(dtype='int'),
                    'Total Games': pd.Series(dtype='int'),
                    'Total Wins': pd.Series(dtype='int'),
                    'Total Losses': pd.Series(dtype='int'),
                    'Total Ties': pd.Series(dtype='int'),
                    'PPG': pd.Series(dtype='float'),})

  for t in teams:
    t_result = {'Team':t, 'Total Points': 0,
                    'Total Games': 0,
                    'Total Wins': 0,
                    'Total Losses': 0,
                    'Total Ties': 0,
                    'PPG':0}
    ranking_df = ranking_df.append(t_result, ignore_index=True)

  # populate ranking dataframe based on schedule
  for index,row in schedule_df.iterrows():
    home_team = row['Home Team']
    away_team = row['Away Team']
    home_score = row['Home Score']
    away_score = row['Away Score']

    ranking_df.loc[(ranking_df['Team'] == home_team), 'Total Games'] += 1
    ranking_df.loc[(ranking_df['Team'] == away_team), 'Total Games'] += 1

    if home_score > away_score: # home win  
      ranking_df.loc[(ranking_df['Team'] == home_team), 'Total Wins'] += 1
      ranking_df.loc[(ranking_df['Team'] == home_team), 'Total Points'] += 3
      ranking_df.loc[(ranking_df['Team'] == away_team), 'Total Losses'] += 1
    elif home_score < away_score: # away win
      ranking_df.loc[(ranking_df['Team'] == away_team), 'Total Wins'] += 1
      ranking_df.loc[(ranking_df['Team'] == away_team), 'Total Points'] += 3
      ranking_df.loc[(ranking_df['Team'] == home_team), 'Total Losses'] += 1
    elif home_score == away_score: # tie
      ranking_df.loc[(ranking_df['Team'] == home_team), 'Total Ties'] += 1
      ranking_df.loc[(ranking_df['Team'] == away_team), 'Total Ties'] += 1
      ranking_df.loc[(ranking_df['Team'] == home_team), 'Total Points'] += 1
      ranking_df.loc[(ranking_df['Team'] == away_team), 'Total Points'] += 1

  ranking_df['PPG'] = ranking_df['Total Points'] / ranking_df['Total Games']

  ranking_df = ranking_df.sort_values(by=['PPG', 'Total Points'], 
                                      ascending=False)
  ranking_df = ranking_df.drop_duplicates(keep='last')
  return ranking_df


# Running Script

In [ ]:
# demo
df = get_scores_from_maxpreps_for_one_day("2/21/2023")
print(df)

In [ ]:
# get all scores
# please note game info of previous seasons/years are not accessible,
# only current season is accessible.

# load historical data
# scores.csv was generated between 11/10/2022 and 2/1/2023
hist_scores = pd.read_csv('/drive/My Drive/scores.csv')

start_date = date(2023, 2, 1)
end_date = date(2023, 3, 12)
new_scores = get_scores_from_maxpreps_for_range(start_date, end_date)

scores = pd.concat([hist_scores, new_scores])
scores['Date'] = pd.to_datetime(scores['Date'])
scores = scores.drop_duplicates(keep='last')
scores.to_csv('/drive/My Drive/scores_2223_03112023.csv', index=False)

In [ ]:
# get all rankings
rankings = get_rankings_from_scores(scores)
rankings.to_csv('/drive/My Drive/rankings_2223_03112023.csv', index=False)

# New Section

# Advanced Method

In [ ]:
# create a new dataset for the scores and rankings
# one shot (not combining)

# combine with historical data

# Test of Script

### analyze game summary page

In [ ]:
single_date =  date(2023, 2, 28)
today = single_date.strftime("%m/%d/%Y")

# maxpreps's homepage
maxpreps_url = 'https://www.maxpreps.com/ca/central-coast-section/soccer/girls/scores/?date={}'.format(today)  # please change the date if needed

# Use requests to retrieve data from a given URL
maxpreps_response = requests.get(maxpreps_url, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

# Parse the whole HTML page using BeautifulSoup
maxpreps_soup = BeautifulSoup(maxpreps_response.text, 'html.parser')


In [ ]:
# find all the div (each div is a game) and print a link in each div
for gamediv in maxpreps_soup.find_all("div", {"class": "contest-box-item"}):#maxpreps_soup.findAll('a', {'class': 'c-c'}):
    try:
        # print(gamediv['href'])
        # print(gamediv)
        print(gamediv.find('a')['href'])
    except KeyError:
        print("no href")
        pass

In [ ]:
# find  details of one game using the link
# game_detail_page = "https://www.maxpreps.com/games/2-28-2023/girls-soccer-winter-22-23/presentation-vs-windsor.htm?c=kWgQYflpLk60D3-K60z9pA" # simple example
game_detail_page = "https://www.maxpreps.com/games/2-22-2023/girls-soccer-winter-22-23/menlo-atherton-vs-mountain-view.htm?c=JPH2i5aDH0yYvdk1bipRfg" # complete example

# Use requests to retrieve data from a given URL
gamedetail_response = requests.get(game_detail_page, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

# Parse the whole HTML page using BeautifulSoup
gamedetail_soup = BeautifulSoup(gamedetail_response.text, 'html.parser')

In [ ]:
# print('Classes of each table:')
# for table in gamedetail_soup.find_all('table'):
#     print(table.get('class'))

def get_text(ele):
  return ele.text if ele is not None else None


def find_city(team_name, summary):
  """ find city of team from summary
  """
  res = re.search(r'{} \((.*?)\)'.format(team_name), summary, re.IGNORECASE)
  if res:
      return res.group(1)




# need to record howm/away information in one row

## create a dataframe from statistics page

In [ ]:
# find info for a game from its details url
def extract_game_info_from_details_page(page_url):
  # Use requests to retrieve data from a given URL
  page_response = requests.get(page_url, 
                               headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

  # Parse the whole HTML page using BeautifulSoup
  page_soup = BeautifulSoup(page_response.text, 'html.parser')

  # find summary
  game_summary = get_text(page_soup.find('p', attrs={'class' : 'contest-description'}))
  # print(game_summary)

  # find school names (names in table are not reliable)
  school_names = [i.text for i in page_soup.find('div', attrs={'class' : 'school-names'}).find_all('a')]
  # print(school_names)

  # analyze target table
  table = page_soup.find('div' , {"data-l-s-c":"box-score"}).find('table', class_='mx-grid boxscore d-b-s post soccer')
  # print(table)

  team_names = []
  team_cities = []
  first_half_scores = []
  second_half_scores = []
  total_scores = []
  shootout_scores = []
  is_winner = []

  try: 
    for r in table.tbody.find_all('tr'):
      # find team name
      team_name = get_text(r.find('th', class_='team first'))
      team_names.append(team_name)
      # team_cities.append(find_city(team_name, game_summary))
      first_half_scores.append(get_text(r.find('td', class_='firsthalf score dw')))
      second_half_scores.append(get_text(r.find('td', class_='secondhalf score dw')))
      total_scores.append(get_text(r.find('td', class_='score total score')))
      shootout_scores.append(get_text(r.find('td', class_='shootout stat tiebreaker dw')))
      is_winner.append(get_text(r.find('td', class_='winner last')))  
  except AttributeError as err:
    print(err)
    pass

  # team_names = [j if i != j else i for i, j in zip(team_names, school_names)]
  team_names = school_names
  team_cities = [find_city(j, game_summary) for j in team_names]
  return team_names, team_cities, first_half_scores, second_half_scores, total_scores, shootout_scores, is_winner, game_summary

game_detail_page1 = "https://www.maxpreps.com/games/2-28-2023/girls-soccer-winter-22-23/presentation-vs-windsor.htm?c=kWgQYflpLk60D3-K60z9pA" # simple example
game_detail_page2 = "https://www.maxpreps.com/games/2-22-2023/girls-soccer-winter-22-23/menlo-atherton-vs-mountain-view.htm?c=JPH2i5aDH0yYvdk1bipRfg" # complete example with shootout
game_detail_page3 = "https://www.maxpreps.com/games/3-2-2023/girls-soccer-winter-22-23/hollister-vs-stone-ridge-christian.htm?c=tv8S33dtGkSS-2ZuVO0vLg"
game_detail_page4 = "https://www.maxpreps.com/games/11-29-2022/girls-soccer-winter-22-23/gateway-vs-oceana.htm?c=70Hrb_EAd0mWPDTpdhiXoQ" # example of missing scores
game_detail_page5 = "https://www.maxpreps.com/games/1-6-2023/girls-soccer-winter-22-23/king-city-vs-st-francis.htm?c=ImFn1XMo2EGTXl-9PYtm0A" # no game result is reported

# print(extract_game_info_from_details_page(game_detail_page1))
# print(extract_game_info_from_details_page(game_detail_page2))
print(extract_game_info_from_details_page(game_detail_page3))
print(extract_game_info_from_details_page(game_detail_page5))

In [ ]:
# extract team info from school profile page
def extract_team_info_from_profile_page(school_profile_url):
  """
  return school name, address, mascot, color, school type, athletic director, phone
  """
  # Use requests to retrieve data from a given URL
  page_response = requests.get(school_profile_url, 
                               headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

  # Parse the whole HTML page using BeautifulSoup
  page_soup = BeautifulSoup(page_response.text, 'html.parser')

  school_name = page_soup.find("h1").string

  dl_data = page_soup.find_all("dd")
  info_list = [dl.text for dl in dl_data]
  
  return tuple([school_name] + info_list)
team_profile_page1 = "https://www.maxpreps.com/ca/mountain-view/mountain-view-spartans"

print(extract_team_info_from_profile_page(team_profile_page1))



In [ ]:
# find team profile page url from game page url
def find_team_profile_pages(game_page_url):
  # Use requests to retrieve data from a given URL
  page_response = requests.get(game_page_url, 
                               headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

  # Parse the whole HTML page using BeautifulSoup
  page_soup = BeautifulSoup(page_response.text, 'html.parser')

  page_urls = []
  for sdiv in page_soup.find_all("div", {"class": "school-names"}):
    for l in sdiv.find_all('a'):
      page_urls.append("https://www.maxpreps.com"+l['href'])
  return page_urls

game_detail_page5 = "https://www.maxpreps.com/games/1-6-2023/girls-soccer-winter-22-23/king-city-vs-st-francis.htm?c=ImFn1XMo2EGTXl-9PYtm0A" # no game result is reported
print(find_team_profile_pages(game_detail_page5))

In [ ]:
from tqdm.notebook import tqdm

# create dataframe of teams

teams_df = pd.DataFrame({'School Name': pd.Series(dtype='str'),
                         'School Alias': pd.Series(dtype='str'),
                        'School Address': pd.Series(dtype='str'),
                        'Mascot': pd.Series(dtype='int'),
                        'School Link': pd.Series(dtype='str'),
                        'Team Link': pd.Series(dtype='str')})

# load schedule df to 
# step 1: find team link from each game details page
# step 2: add a record of team info
# step 3: dedupe 
# note: moving forward we don't need to parse school info from game details page

hist_scores_df = pd.read_csv('/drive/My Drive/scores_2223_03182023.csv')
for index, row in tqdm(hist_scores_df.iterrows(), total=hist_scores_df.shape[0]):
    game_url = row['Web Link']
    team_profile_urls = find_team_profile_pages(game_url)
    for u in team_profile_urls:
        school_profile_url = u.removesuffix("soccer/girls/winter/schedule/")
        info_list = extract_team_info_from_profile_page(school_profile_url)
        try:
          # pd.concat([new_row,df.loc[:]]).reset_index(drop=True)
          m_result = pd.DataFrame({'School Name':info_list[0], 
                      'School Alias':"", 
                      'School Address':info_list[1], 
                      'Mascot':info_list[2], 
                      'School Link':school_profile_url,
                      'Team Link':u }, index=[0])
          teams_df = pd.concat([m_result, teams_df.loc[:]]).reset_index(drop=True)
          # teams_df = teams_df.append(m_result, ignore_index=True)
        except IndexError as err:
          print(err)
          print(school_profile_url)
          print(u)

teams_df = teams_df.drop_duplicates(keep='last')
# teams_df.to_csv('/drive/My Drive/teams_2223_03182023.csv', index=False)
print(teams_df)


In [ ]:
# create dateframe of game information
def create_game_info_dataframe_from_stats_page(page_url, today):
  """ create game info dataframe from a stats page

  today: Date object
  
  note: details page has no date
  return dataframe

  """
  
  
  # Use requests to retrieve data from a given URL
  page_response = requests.get(page_url, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

  # Parse the whole HTML page using BeautifulSoup
  page_soup = BeautifulSoup(page_response.text, 'html.parser')

  # verify the date
  title_date = datetime.strptime(page_soup.title.text.split('|')[1].strip().split()[0], '%m/%d/%Y').date()
  # print(page_soup.title.text.split('|')[1].strip().split()[0])
  # print(title_date)
  # print(today)

  if title_date == today:
      print('Found Games for {}'.format(title_date))
      schedule_df = pd.DataFrame({'Date': pd.Series(dtype='datetime64[ns]'),
                                'Home Team': pd.Series(dtype='str'),
                                'Home City': pd.Series(dtype='str'),
                                'Home Score': pd.Series(dtype='int'),
                                'Away Team': pd.Series(dtype='str'),
                                'Away City': pd.Series(dtype='str'),
                                'Away Score': pd.Series(dtype='int'),
                                'Game Summary': pd.Series(dtype='str'),
                                'Web Link': pd.Series(dtype='str')})
      # find all the div (each div is a game) and print a link in each div
      for gamediv in page_soup.find_all("div", {"class": "contest-box-item"}):#maxpreps_soup.findAll('a', {'class': 'c-c'}):
          try:
              game_detail_url = gamediv.find('a')['href']
              print(game_detail_url)
              team_names, team_cities, first_half_scores, second_half_scores, total_scores, shootout_scores, is_winner, game_summary = extract_game_info_from_details_page(game_detail_url)
              if len(total_scores)==2:
                  if shootout_scores[0] == None or shootout_scores[1]==None:
                    m_result = {'Date':today, 
                                'Home Team':team_names[1], 
                                'Home City':team_cities[1], 
                                'Home Score':total_scores[1], 
                                'Away Team':team_names[0],
                                'Away City':team_cities[0], 
                                'Away Score':total_scores[0],
                                'Game Summary': game_summary,
                                'Web Link': game_detail_url}
                  else:
                    m_result = {'Date':today, 
                                'Home Team':team_names[1], 
                                'Home City':team_cities[1], 
                                'Home Score':shootout_scores[1], 
                                'Away Team':team_names[0],
                                'Away City':team_cities[0], 
                                'Away Score':shootout_scores[0],
                                'Game Summary': game_summary,
                                'Web Link': game_detail_url}
                  schedule_df = schedule_df.append(m_result, ignore_index=True)
          except (KeyError, TypeError) as err:
              print(err)
              pass

      return schedule_df
  else:
      print('No Games for {}'.format(today))
    



In [ ]:
today = date(2022, 11, 29)
maxpreps_url = 'https://www.maxpreps.com/ca/central-coast-section/soccer/girls/scores/?date={}'.format(today.strftime("%m/%d/%Y")) 
print(maxpreps_url)


print(create_game_info_dataframe_from_stats_page(maxpreps_url, today))

In [ ]:
# reprocess all season data
from tqdm.notebook import tqdm

start_date = date(2022, 11, 1)
end_date = date(2023, 3, 18)

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

dfs = []
for today in tqdm(daterange(start_date, end_date)):
    # print('https://www.maxpreps.com/ca/central-coast-section/soccer/girls/scores/?date={}'.format(today.strftime("%m/%d/%Y")) )
    scores_df = create_game_info_dataframe_from_stats_page(
        'https://www.maxpreps.com/ca/central-coast-section/soccer/girls/scores/?date={}'.format(today.strftime("%m/%d/%Y")) ,
        today)
    if scores_df is not None:
      print('Found {} games for {}'.format(len(scores_df), today))
    dfs.append(scores_df)

# see pd.concat documentation for more info
if len(dfs)>0:
  scores_df = pd.concat(dfs)
  scores_df = scores_df.drop_duplicates(keep='last')
  print(scores_df)


# scores['Date'] = pd.to_datetime(scores['Date'])
# scores = scores.drop_duplicates(keep='last')
scores_df.to_csv('/drive/My Drive/scores_2223_03182023.csv', index=False)

In [ ]:
# get all rankings
rankings = get_rankings_from_scores(scores_df)
rankings.to_csv('/drive/My Drive/rankings_2223_03182023.csv', index=False)